In [1]:
%matplotlib inline
%load_ext autoreload

%autoreload 2
import tensorflow as tf
tf.enable_eager_execution()
tf.executing_eagerly()

True

In [2]:
# !pip install ../forks/tensortrade_source/ -U

In [3]:
import os
import sys
import warnings
import numpy
import ccxt
import pandas as pd
import numpy as np
from time import time
import math
import cProfile as cp
import matplotlib.pyplot as plt
import pickle

from gym.spaces import Space, Box

config = './neat.config'

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 

sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.abspath('')+"\\tensortrade")

fig = plt.figure()
fig.patch.set_facecolor('xkcd:white')

<Figure size 432x288 with 0 Axes>

**Load Data**

In [4]:

data_file ='./data/processed/binance/btc_usdt_1h.csv' 
df = pd.read_csv(data_file, index_col=[0])


# number of days we want to pull from the dataframe 
days_of_data = 365

# number of data frames (our DF is in 1h timesteps)
frames = days_of_data * 24 * 5
# frames = len(df)
train_test_percentage = 0.4

x_train = int(frames * (1 - train_test_percentage))
x_test = int(frames - x_train)

df_train = df[-frames:(-x_test - 1)]
df_test = df[-x_test:]
print("Friend Shape", df.shape)
print('train shape', df_train.shape)
print('test shape', df_test.shape)
print('columns', df.columns)

Friend Shape (180236, 12)
train shape (26279, 12)
test shape (17520, 12)
columns Index(['open', 'high', 'low', 'close', 'volume', 'momentum_rsi', 'adx',
       'adx_long', 'adx_pos', 'adx_neg', 'trend_ema_fast', 'trend_ema_slow'],
      dtype='object')


In [5]:
df_test.head()

,open,high,low,close,volume,momentum_rsi,adx,adx_long,adx_pos,adx_neg,trend_ema_fast,trend_ema_slow
index,,,,,,,,,,,,
2019-08-28 03:35:00,10148.20,10155.65,10140.00,10140.49,103.707956,34.890237,19.680116,9.632237,15.942283,24.099556,10153.780045,10155.268113
2019-08-28 03:40:00,10140.74,10148.78,10125.01,10133.83,73.728054,30.560787,19.729527,9.551850,14.434463,27.784685,10150.710807,10153.680105
2019-08-28 03:45:00,10135.11,10145.87,10120.00,10120.01,102.866752,23.560759,20.578936,9.524416,12.994025,26.944578,10145.987606,10151.186023
2019-08-28 03:50:00,10120.02,10133.51,10116.34,10120.03,55.034065,23.589984,21.604012,9.514225,12.128909,26.569855,10141.994128,10148.878170
2019-08-28 03:55:00,10120.03,10130.00,10119.02,10126.17,48.912488,32.704088,22.726318,9.516439,11.597170,25.405014,10139.559647,10147.196083


**Load Previous Winner**

In [6]:
# winner = "winner-1751.bin"
# with open("winners/{}".format(winner), "rb") as f:
#     winner = pickle.load(f)

**Train against training data**

In [ ]:
import tensortrade
from neat_trading_strategy import NeatTradingStrategy as TradingStrategy
from neat_reward_strategy import NeatRewardStrategy as ProfitStrategy
from tensortrade.actions import DiscreteActions
from tensortrade.exchanges.simulated import SimulatedExchange as Exchange
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline
from tensortrade.environments import TradingEnvironment as Environment

print('fin imports')
normalize = MinMaxNormalizer(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

reward_scheme = ProfitStrategy()
action_scheme = DiscreteActions(n_actions=5, instrument='BTC/USDT')
print('fin strats')

exchange = Exchange(data_frame=df_train,
                    pretransform = True,
                    base_instrument='USDT', 
                    commission_percent=0.075,
                    window_size=1,
                    max_allowed_slippage_percent=3.0,
                    min_order_amount=1E-4,
                    min_trade_amount=1E-4,
                    observation_columns = df_train.columns
                   )
print('fin exchange')

environment = Environment(exchange=exchange,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline)
print('fin environment')
print('')

segments_in_day = 288
days = 14

exchange._balance = 100
strategy = TradingStrategy(environment=environment,
                           pop_size= 500,
                           initial_connectin='full_nodirect',
                           max_stagnation= 10,
                           neat_config=config, 
                           watch_genome_evaluation=True,
                           only_show_profitable=True,
                           data_frame_window = segments_in_day * days,
                           disable_full_evaluation = True
                          )

print("Running through ", strategy._data_frame_window, ' steps')
# cp.run("performance, winner, stats = strategy.run(generations=20)", 'evolution_stats')
performance, winner, stats = strategy.run(generations=100)

fin imports
fin strats
fin exchange
fin environment

Running through  4032  steps

 ****** Running generation 0 ****** 

Population's average fitness: 239.71141 stdev: 1465.31367
Best fitness: 10401.57446 - size: (13, 181) - species 2 - id 492
Average adjusted fitness: 0.028
Mean genetic distance 4.334, standard deviation 0.383
Population of 500 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   204   9563.6    0.018     0
     2    0   296  10401.6    0.038     0
Total extinctions: 0
Generation time: 9404.146 sec
Saving checkpoint to neat-checkpoint-0

 ****** Running generation 1 ****** 

Population's average fitness: 459.13784 stdev: 1960.71202
Best fitness: 10159.46415 - size: (13, 174) - species 1 - id 570
Average adjusted fitness: 0.042
Mean genetic distance 4.115, standard deviation 0.404
Population of 500 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====


**Evaluate against full dataset**

In [ ]:
 _test_data = df_test[-280*14:(-280*9)]
exchange = Exchange(data_frame=_test_data,
                    pretransform = True,
                    base_instrument='USDT', 
                    commission_percent=0.075,
                    window_size=1,
                    max_allowed_slippage_percent=3.0,
                    min_order_amount=1E-4,
                    min_trade_amount=1E-4,
                    observation_columns = df_test.columns
                   )
environment = Environment(exchange=exchange,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline)

environment.reset()
# from neat_trading_strategy import NeatTradingStrategy as TradingStrategy
# exchange.balance = 100000
strategy = strategy = TradingStrategy(environment=environment, 
                           neat_config=config, 
                           watch_genome_evaluation=True,
                           only_show_profitable=False,
                           full_evaluation=True
                          )
# TradingStrategy(environment=environment, watch_genome_evaluation=True, full_evaluation = True, neat_config=config)

In [ ]:
exchange._balance = 100
print("Starting Worth: {}".format(exchange.net_worth, exchange.balance))
strategy.eval_genome(winner, strategy._config)

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
exchange.performance.tail()

In [ ]:
exchange.trades.tail()

In [ ]:
ax = plt.subplot(211)
exchange.performance.net_worth.plot(figsize=(22,8), label='net worth', title='Net Worth')
exchange.performance['com'] = exchange.trades.price*0.00075
ax2 = plt.subplot(212, sharex=ax)
exchange.performance.com.plot(title='Commissions')
plt.legend()

In [ ]:
wins = 0
losses = 0 
ax = plt.subplot(311)

exchange._price_history.plot(figsize=(22,8), label='actual price')
print(_test_data.columns)
_test_data.trend_ema_fast.plot(label='fast ema')
_test_data.trend_ema_slow.plot(label='slow ema')

# print(exchange._price_history.head())
# print(exchange.trades.head())
# print(exchange.performance.head())
for idx,trade in exchange.trades.iterrows():
    price_at_trade = exchange._price_history[trade.step-1]
    if trade.type.is_buy:
        plt.plot(trade.step-1 , price_at_trade, "b^")
    elif trade.type.is_sell:
        the_buy_step = exchange.trades.iloc[idx-1]
        profit = trade.price - the_buy_step.price
        if profit < 0:
            losses +=1
            color = '#d62728'
            plt.axvspan(the_buy_step.step-1, trade.step-1, alpha= 0.5, facecolor='red')
            plt.plot(trade.step-1, price_at_trade, "rD")

        elif profit >= 0:
            wins +=1
            color = '#2ca02c'
            plt.axvspan(the_buy_step.step-1, trade.step-1, alpha= 0.5, facecolor='green')
            plt.plot(trade.step-1, price_at_trade, "gD")

adx = plt.subplot(312, sharex=ax)
_test_data.adx.plot(label='adx')
_test_data.adx_pos.plot(label='pos')
_test_data.adx_neg.plot(label='neg')
_test_data.adx_long.plot(label='long')

_test_data.momentum_rsi.plot(label='rsi')

ax2 = plt.subplot(313, sharex=adx)
exchange.performance.net_worth.plot(title='Net Worth')
ax.legend()

ax.set_title("Wins:{} Losses:{} Win rate {}%".format(wins, losses, round((wins/(wins+losses))*100, 4) ))


In [ ]:
import visualize

visualize.plot_stats(stats, ylog=False, view=True)


In [ ]:
visualize.plot_species(stats, view=True)

In [ ]:
visualize.draw_net(strategy._config, winner, False)


**Save Strategy**

In [ ]:
# winner.key
with open("winners/winner-{}.bin".format(winner.key), "wb") as f:
    pickle.dump(winner, f, 2)
with open("winners/winner-{}_bak.bin".format(winner.key), "wb") as f:
    pickle.dump(winner, f, 2)